# Data & Technology: SCADA Assistant + Digital Asset Management

In [ ]:
import pandas as pd
scada = pd.read_csv('scada_events.csv', parse_dates=['timestamp'])
assets = pd.read_csv('digital_assets.csv')
print(scada.head()); print(assets.head())

## KPIs

In [ ]:
print('Total events:', len(scada))
print('Alarms:', (scada.event_type=='ALARM').sum())
print('Last event at:', scada['timestamp'].max())

## Rule-based Q&A

In [ ]:
def assistant(query):
    q = query.lower()
    if 'last alarm' in q:
        a = scada[scada.event_type=='ALARM'].sort_values('timestamp', ascending=False).head(5)
        return a[['timestamp','equipment','reading_value']]
    if 'events for' in q:
        eq = q.split('events for')[-1].strip()
        return scada[scada['equipment'].str.lower().str.contains(eq)].tail(10)
    if 'search asset' in q:
        atype = 'pump' if 'pump' in q else None
        loc = 'Plant-1' if 'plant-1' in q else None
        df = assets.copy()
        if atype: df = df[df['asset_type'].str.lower()==atype]
        if loc: df = df[df['location']==loc]
        return df.head(10)
    return 'Try: last alarm | events for <equipment> | search asset <type> in <location>'
print(assistant('last alarm'))

## Intent Classification (TF-IDF + Logistic Regression)

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
data = pd.DataFrame({
    'text':[
        'show last alarm','list last alarms','last alarm pump-03',
        'events for pump-01','show events for valve-02','events for sensor-05',
        'search asset pump in plant-1','search asset valve in station-b','find assets sensor plant-2'
    ],
    'intent':['last_alarm','last_alarm','last_alarm','events_for','events_for','events_for','asset_search','asset_search','asset_search']
})
X_train, X_test, y_train, y_test = train_test_split(data['text'], data['intent'], test_size=0.33, random_state=42)
vec = TfidfVectorizer(); clf = LogisticRegression(max_iter=1000)
clf.fit(vec.fit_transform(X_train), y_train)
print('Test accuracy:', clf.score(vec.transform(X_test), y_test))
print('Predict:', clf.predict(vec.transform(['show alarms for pump-03']))[0])